In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark
!pip install haversine

In [ ]:
#5.Please find the list of foreshocks and aftershocks (within 20 km and in 24 hours) for top ten earthquakes between years 1990 and 2019.
from pyspark import SparkContext
from datetime import datetime, timedelta
from haversine import haversine, Unit
sc=SparkContext.getOrCreate()
earthquakeRDD = sc.textFile("EarthquakeDataset.txt")
earthquakeRDD = earthquakeRDD.filter(lambda line: "Event ID" not in line) #header satırını kaldırdık.

def splitAndTake(line):
  arr = line.split("\t") #delimiter olarak tab kullan
  enlem = float(arr[4]) #noktali degerleri float'a cevirdik.
  boylam = float(arr[5])
  buyukluk = float(arr[7])
  yer = arr[14]
  tarihSaat = arr[2] + ' ' + arr[3] #tarih ve saati birlestirdik.
  tarihSaat = tarihSaat[:-6] #datada 73 saniye gibi gecersiz degerler oldugu icin saatin sonundaki saniye ve saliseleri kaldırdık.(ornek:11300'deki satir: 10:03:73.00)
  tarihSaat = datetime.timestamp(datetime.strptime(tarihSaat,'%Y.%m.%d %H:%M')) #tarihler arasında kolay islem yapabilmek icin timestamp formatına cevirdik.
  return (tarihSaat, enlem, boylam, buyukluk, yer ) 

def distance(deprem1,deprem2): #enlem ve boylam'dan 20 km'lik alanı hesaplamak icin kullanıyoruz.
  enlemboylam1 = (deprem1[1],deprem1[2])
  enlemboylam2 = (deprem2[1],deprem2[2])
  return (haversine(enlemboylam1,enlemboylam2),deprem1,deprem2)

def timeDif(item1,item2,item3): #item1-->deprem1, item2-->deprem2, item3-->uzaklik parametrelerini cagararak iki zaman arasındaki farki hesaplayip, yazdirdik.
  zamanFarki = item1[0] - item2[0]
  zamanFarki = zamanFarki / (60 * 60)
  return (item1,item2,item3,zamanFarki)

earthquakeRDD = earthquakeRDD.map(lambda a : splitAndTake(a))
earthquakeRDD = earthquakeRDD.filter(lambda line: line[0] >= datetime.timestamp(datetime.strptime("1990.01.01 00:00",'%Y.%m.%d %H:%M'))) #1990'dan kucuk degerleri kaldırdık.
earthquakeRDD = earthquakeRDD.filter(lambda line: line[0] <= datetime.timestamp(datetime.strptime("2019.12.31 23:59",'%Y.%m.%d %H:%M'))) #2019'dan buyuk degerleri kaldırdık.
earthquakeRDD = earthquakeRDD.sortBy(lambda b: b[3], ascending=False) #buyuklugune gore depremleri sıralıyoruz.
topTenRDD = sc.parallelize(earthquakeRDD.take(10)) #ilk 10 buyuk depremi alıyoruz.
earthquakeRDD1 = earthquakeRDD.cartesian(topTenRDD) #ilk 10 deprem ile tum depremleri aynı satira getirip, ikili tuple yapıyoruz.
earthquakeRDD1 = earthquakeRDD1.sortBy(lambda a: a[0]) #kontrol amacli sıralıyoruz.
earthquakeRDD1 = earthquakeRDD1.filter(lambda line: line[0] != line[1]) #kartezyen nedeni ile olusan duplike satirlari kaldırıyoruz.
earthquakeRDD1 = earthquakeRDD1.map(lambda z: distance(z[0],z[1])) #distance fonksiyonun cagararak uzaklik(km) kolonunu ekledik.
earthquakeRDD1 = earthquakeRDD1.filter(lambda z: z[0] <= 20 ) #20km'den buyuk depremleri filtreliyoruz.
earthquakeRDD1 = earthquakeRDD1.map(lambda z: (z[2],z[1],z[0])) #kolon yerlerini degistirdik.
earthquakeRDD1 = earthquakeRDD1.map(lambda z: timeDif(z[0],z[1],z[2])) #iki deprem arasındaki zaman farkini hesaplayip yazdırdık.
earthquakeRDD1 = earthquakeRDD1.filter(lambda z: z[3] <= 12 ) #24 saatlik karsılastirma istendigi icin, 12 saat oncesi ve sonrası olarak datayi filtredik.
earthquakeRDD1 = earthquakeRDD1.filter(lambda z: z[3] >= -12 ) #24 saatlik karsılastirma istendigi icin, 12 saat oncesi ve sonrası olarak datayi filtredik.
earthquakeRDD1 = earthquakeRDD1.sortBy(lambda a: a[0]) #datayi buyuk depremlere gore sıraladık.

print("Top ten:") #bilgi amacli ilk 10 buyuk depremi gostermek icin kullandık.
for r in earthquakeRDD.take(10):print(datetime.fromtimestamp(r[0]).strftime('%Y.%m.%d %H:%M'),r[1],r[2],r[3],r[4])

print("\nList of foreshocks and aftershocks (within 20 km and in 24 hours):") #Timestamp'i normal date'e cevirerek, sirasi ile asil deprem bilgisi, varsa oncu/artci bilgisi, uzakligi ve zaman farkini gosterdik.
for d in earthquakeRDD1.collect():
  print(datetime.fromtimestamp(d[0][0]).strftime('%Y.%m.%d %H:%M'),d[0][1],d[0][2],d[0][3],d[0][4],datetime.fromtimestamp(d[1][0]).strftime('%Y.%m.%d %H:%M'),d[1][1],d[1][2],d[1][3],d[1][4],d[2],d[3] )

#ANSWER: Toplam 166 oncu/artci cıktı.
#Top ten:
#1999.08.17 00:01 40.76 29.97 7.4 BASISKELE (KOCAELI) [North East  2.0 km]
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km]
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km]
#1992.03.13 17:18 39.72 39.63 6.8 GUNEBAKAN- (ERZINCAN) [South West  1.7 km]
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ)
#2008.07.15 03:26 35.7948 27.8798 6.4 AKDENIZ
#2003.05.01 00:27 39.01 40.46 6.4 KURTULUS- (BINGOL) [South West  4.3 km]
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI
#1998.06.27 13:55 36.96 35.52 6.3 HASANBEYLI-SARICAM (ADANA) [South 10.1 km]
#2011.04.01 13:29 35.5138 26.5798 6.2 AKDENIZ
#
#List of foreshocks and aftershocks (within 20 km and in 24 hours):
#1992.03.13 17:18 39.72 39.63 6.8 GUNEBAKAN- (ERZINCAN) [South West  1.7 km] 1992.03.14 01:24 39.63 39.69 4.6 KUCUKKADAGAN- (ERZINCAN) [South East  1.7 km] 11.248114686422076 -8.1
#1998.06.27 13:55 36.96 35.52 6.3 HASANBEYLI-SARICAM (ADANA) [South 10.1 km] 1998.06.27 14:07 36.81 35.61 4.2 AKPINAR-YUREGIR (ADANA) [South East  1.0 km] 18.50051630107028 -0.2
#1998.06.27 13:55 36.96 35.52 6.3 HASANBEYLI-SARICAM (ADANA) [South 10.1 km] 1998.06.27 15:12 36.97 35.7 3.5 COKCAPINAR-CEYHAN (ADANA) [South West  2.1 km] 16.030744805064483 -1.2833333333333334
#1998.06.27 13:55 36.96 35.52 6.3 HASANBEYLI-SARICAM (ADANA) [South 10.1 km] 1998.06.27 18:54 37.01 35.68 4.0 CAKALDERE-CEYHAN (ADANA) [North 2.1 km] 15.26032201154447 -4.983333333333333
#1998.06.27 13:55 36.96 35.52 6.3 HASANBEYLI-SARICAM (ADANA) [South 10.1 km] 1998.06.27 20:03 36.97 35.65 3.7 CAKALDERE-CEYHAN (ADANA) [South West  3.8 km] 11.603277682169173 -6.133333333333334
#1998.06.27 13:55 36.96 35.52 6.3 HASANBEYLI-SARICAM (ADANA) [South 10.1 km] 1998.06.27 20:50 36.84 35.56 4.1 CATALPINAR-YUREGIR (ADANA) [North West  2.0 km] 13.809333024749346 -6.916666666666667
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 17:17 40.75 31.1 5.2 AYDINPINAR- (DUZCE) [South West  1.6 km] 9.333295549291424 -0.3333333333333333
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 17:18 40.74 31.05 5.4 HAMAMUSTU-GOLYAKA (DUZCE) [South East  1.2 km] 13.480024723001737 -0.35
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 17:23 40.76 31.16 5.1 KALEDIBI- (DUZCE) [South West  1.3 km] 4.762943204952413 -0.43333333333333335
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 17:27 40.75 31.4 5.2 DARIYERIBAKACAK-KAYNASLI (DUZCE) [East 1.2 km] 16.04490190932572 -0.5
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 17:52 40.76 31.22 4.8 OZYANIK- (DUZCE) [South West  1.4 km] 2.37809460508355 -0.9166666666666666
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 17:54 40.77 31.26 4.7 FINDIKLI-KAYNASLI (DUZCE) [South East  0.4 km] 5.372628062109872 -0.95
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 17:57 40.74 31.4 4.8 DARIYERIBAKACAK-KAYNASLI (DUZCE) [South East  1.6 km] 16.007528449932316 -1.0
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 18:14 40.75 31.36 5.0 YESILTEPE-KAYNASLI (DUZCE) [North West  2.0 km] 12.68540188441602 -1.2833333333333334
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 18:24 40.76 31.11 4.7 AYDINPINAR- (DUZCE) [South East  0.3 km] 8.71236426558482 -1.45
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 19:00 40.75 31.09 4.2 GOLORMANI- (DUZCE) [South East  1.2 km] 10.17022884813415 -2.05
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 19:07 40.77 31.2 4.5 GUVEN- (DUZCE) [South West  0.5 km] 3.4405523016291073 -2.1666666666666665
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 19:16 40.74 31.38 4.7 YESILTEPE-KAYNASLI (DUZCE) [North East  0.2 km] 14.322526012679635 -2.316666666666667
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 20:05 40.74 31.0 4.5 YUNUSEFENDI-GOLYAKA (DUZCE) [South West  1.1 km] 17.692530679497327 -3.1333333333333333
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 20:54 40.76 31.38 4.5 DARIYERIBAKACAK-KAYNASLI (DUZCE) [North West  1.2 km] 14.492025526783902 -3.95
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 21:25 40.74 31.03 4.0 CAMLIBEL-GOLYAKA (DUZCE) [North East  0.9 km] 15.165027255900025 -4.466666666666667
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 21:38 40.76 31.1 4.4 AYDINPINAR- (DUZCE) [South West  0.8 km] 9.529259951499386 -4.683333333333334
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 22:21 40.75 31.39 4.8 DARIYERIBAKACAK-KAYNASLI (DUZCE) [East 0.4 km] 15.204601617917454 -5.4
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 22:38 40.76 31.3 4.1 TAVAK-KAYNASLI (DUZCE) [North 1.7 km] 7.900821302570403 -5.683333333333334
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 22:49 40.75 31.05 4.2 HAMAMUSTU-GOLYAKA (DUZCE) [North East  1.0 km] 13.524798774512725 -5.866666666666666
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.12 23:11 40.73 31.41 4.0 YESILTEPE-KAYNASLI (DUZCE) [South East  2.9 km] 16.88794281049278 -6.233333333333333
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.13 00:54 40.72 31.0 4.7 BAKACAK-GOLYAKA (DUZCE) [South East  2.4 km] 17.834390497226803 -7.95
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.13 01:02 40.75 31.4 4.3 DARIYERIBAKACAK-KAYNASLI (DUZCE) [East 1.2 km] 16.04490190932572 -8.083333333333334
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.13 01:57 40.75 31.35 4.0 MURATBEY-KAYNASLI (DUZCE) [North East  2.3 km] 11.846436706547752 -9.0
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.13 02:59 40.75 31.15 4.2 AKTARLA- (DUZCE) [North East  0.6 km] 5.175492018150637 -10.033333333333333
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.13 03:27 40.77 31.37 4.0 DARIYERIHASANBEY-KAYNASLI (DUZCE) [South East  2.4 km] 13.883695799683151 -10.5
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.13 03:57 40.75 31.12 4.2 AYDINPINAR- (DUZCE) [South East  1.7 km] 7.6630489125649754 -11.0
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.13 04:10 40.74 31.38 4.2 YESILTEPE-KAYNASLI (DUZCE) [North East  0.2 km] 14.322526012679635 -11.216666666666667
#1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km] 1999.11.13 04:21 40.75 31.33 4.0 MURATBEY-KAYNASLI (DUZCE) [North 1.8 km] 10.17022884813415 -11.4
#2003.05.01 00:27 39.01 40.46 6.4 KURTULUS- (BINGOL) [South West  4.3 km] 2003.05.01 00:41 39.0 40.37 4.2 KARTAL- (BINGOL) [West 3.8 km] 7.855875748889303 -0.23333333333333334
#2003.05.01 00:27 39.01 40.46 6.4 KURTULUS- (BINGOL) [South West  4.3 km] 2003.05.01 01:01 39.01 40.33 3.5 YAZGULU- (BINGOL) [South East  2.4 km] 11.232336118858534 -0.5666666666666667
#2003.05.01 00:27 39.01 40.46 6.4 KURTULUS- (BINGOL) [South West  4.3 km] 2003.05.01 01:14 38.99 40.38 3.5 KARTAL- (BINGOL) [South West  3.0 km] 7.26208339164967 -0.7833333333333333
#2003.05.01 00:27 39.01 40.46 6.4 KURTULUS- (BINGOL) [South West  4.3 km] 2003.05.01 01:21 39.02 40.49 3.5 KURTULUS- (BINGOL) [South West  1.5 km] 2.820346036262993 -0.9
#2003.05.01 00:27 39.01 40.46 6.4 KURTULUS- (BINGOL) [South West  4.3 km] 2003.05.01 01:31 38.91 40.39 3.7 BILALOGLU- (BINGOL) [South West  1.4 km] 12.660001513974011 -1.0666666666666667
#2003.05.01 00:27 39.01 40.46 6.4 KURTULUS- (BINGOL) [South West  4.3 km] 2003.05.01 01:48 39.05 40.48 3.7 OGULDERE- (BINGOL) [North East  0.5 km] 4.771522537525412 -1.35
#2003.05.01 00:27 39.01 40.46 6.4 KURTULUS- (BINGOL) [South West  4.3 km] 2003.05.01 01:57 39.04 40.5 3.5 OGULDERE- (BINGOL) [South East  2.0 km] 4.8028633670231375 -1.5
#2003.05.01 00:27 39.01 40.46 6.4 KURTULUS- (BINGOL) [South West  4.3 km] 2003.05.01 06:34 39.07 40.26 4.2 GUNGOREN- (BINGOL) [South West  3.0 km] 18.51686852065446 -6.116666666666666
#2003.05.01 00:27 39.01 40.46 6.4 KURTULUS- (BINGOL) [South West  4.3 km] 2003.05.01 06:53 39.07 40.43 3.5 SUDUGUNU- (BINGOL) [North West  2.1 km] 7.157150983400519 -6.433333333333334
#2003.05.01 00:27 39.01 40.46 6.4 KURTULUS- (BINGOL) [South West  4.3 km] 2003.05.01 08:40 39.01 40.42 3.5 KARTAL- (BINGOL) [North East  1.7 km] 3.456103687107886 -8.216666666666667
#2003.05.01 00:27 39.01 40.46 6.4 KURTULUS- (BINGOL) [South West  4.3 km] 2003.05.01 09:35 38.97 40.52 4.1 AGACYOLU- (BINGOL) [North West  0.7 km] 6.831809140992412 -9.133333333333333
#2003.05.01 00:27 39.01 40.46 6.4 KURTULUS- (BINGOL) [South West  4.3 km] 2003.05.01 10:31 38.98 40.36 3.9 GOKCEKANAT- (BINGOL) [North East  3.3 km] 9.263565311232398 -10.066666666666666
#2003.05.01 00:27 39.01 40.46 6.4 KURTULUS- (BINGOL) [South West  4.3 km] 2003.05.01 11:51 39.07 40.46 3.6 SUDUGUNU- (BINGOL) [North East  3.0 km] 6.671704814012267 -11.4
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 10:48 38.7513 43.5952 5.1 ILIKAYNAK- (VAN) [West 1.7 km] 16.32358541905569 -0.11666666666666667
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 10:52 38.8157 43.3382 4.9 ESENPINAR- (VAN) [North East  1.8 km] 12.257737867923653 -0.18333333333333332
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 10:56 38.8137 43.4462 5.6 GEDIKBULAK- (VAN) [South East  2.4 km] 10.728734826667637 -0.25
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 11:07 38.7508 43.2342 3.9 DAGONU- (VAN) [South West  1.5 km] 15.684218778314074 -0.43333333333333335
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 11:27 38.837 43.3102 4.2 HALKALI- (VAN) [North West  1.1 km] 15.561153367460609 -0.7666666666666667
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 11:29 38.7462 43.281 3.8 GUVENCLI- (VAN) [North West  3.2 km] 11.613689298477802 -0.8
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 11:32 38.8022 43.254 5.7 YESILSU- (VAN) [North West  4.0 km] 16.3226214812768 -0.85
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 11:44 38.739 43.298 4.3 GUVENCLI- (VAN) [West 1.6 km] 9.999849065031574 -1.05
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 11:47 38.81 43.3 3.5 HALKALI- (VAN) [South West  2.7 km] 13.788205905806128 -1.1
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 11:54 38.73 43.2 3.9 OZYURT- (VAN) [South West  2.0 km] 18.330160899857294 -1.2166666666666666
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 12:03 38.7762 43.197 4.2 OZYURT- (VAN) [North West  3.7 km] 19.539994074508733 -1.3666666666666667
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 12:17 38.7707 43.6007 3.9 YUKARIGOLALAN- (VAN) [North East  2.2 km] 17.347920136103802 -1.6
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 12:23 38.8772 43.4 3.9 KUMLUCA- (VAN) [North 3.9 km] 17.3726052728207 -1.7
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 12:42 38.7287 43.192 4.3 OZYURT- (VAN) [South West  2.5 km] 19.01648362645068 -2.0166666666666666
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 12:53 38.827 43.3917 3.6 KUMLUCA- (VAN) [South West  1.8 km] 11.882817716146771 -2.2
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 12:56 38.7588 43.3375 4.5 SAHGELDI- (VAN) [South West  1.5 km] 7.623503735312662 -2.25
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 13:17 38.8508 43.3263 4.3 HALKALI- (VAN) [North East  2.5 km] 16.173123352896106 -2.6
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 13:22 38.7562 43.5078 3.6 MEYDANCIK- (VAN) [South 1.4 km] 9.253921970809936 -2.683333333333333
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 13:33 38.8275 43.2453 3.6 HALKALI- (VAN) [West 6.3 km] 18.602461458406985 -2.8666666666666667
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 13:46 38.7468 43.2487 4.1 DAGONU- (VAN) [South 1.5 km] 14.362612909265488 -3.0833333333333335
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 13:47 38.7695 43.5735 4.0 YUKARIGOLALAN- (VAN) [North West  0.5 km] 15.081504578396485 -3.1
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 14:52 38.7547 43.2227 3.8 OZYURT- (VAN) [North East  1.4 km] 16.751421036449507 -4.183333333333334
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 15:06 38.8907 43.3432 3.5 HALKALI- (VAN) [North East  7.2 km] 19.741981019975288 -4.416666666666667
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 15:57 38.7462 43.3162 4.6 GUVENCLI- (VAN) [North 0.8 km] 8.68004988282629 -5.266666666666667
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 16:05 38.752 43.553 4.6 ASAGIGOLALAN- (VAN) [North West  1.5 km] 12.78375201680919 -5.4
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 16:38 38.7342 43.509 4.2 AKCAOREN- (VAN) [North East  1.2 km] 8.623170537794595 -5.95
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 17:14 38.716 43.4278 3.5 OCAKLI- (VAN) [South East  2.0 km] 1.5680278614863201 -6.55
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 17:27 38.6873 43.2647 3.5 ATMACA- (VAN) [South East  1.3 km] 13.242973126613377 -6.766666666666667
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 17:32 38.7235 43.254 3.7 ATMACA- (VAN) [North West  2.8 km] 13.622603780878954 -6.85
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 18:10 38.7023 43.2948 5.1 GOLLU- (VAN) [South West  2.6 km] 10.298885226773185 -7.483333333333333
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 18:27 38.665 43.3935 4.4 ASIT- (VAN) [North 3.1 km] 6.431079847906952 -7.766666666666667
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 18:30 38.7315 43.376 3.5 TABANLI- (VAN) [South East  2.3 km] 3.2450162613513536 -7.816666666666666
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 18:53 38.776 43.3507 4.9 SAHGELDI- (VAN) [North East  0.7 km] 8.02969241422413 -8.2
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 19:06 38.7852 43.2703 4.9 YESILSU- (VAN) [North West  2.0 km] 14.124634380021034 -8.416666666666666
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 19:43 38.735 43.23 4.5 OZYURT- (VAN) [South East  1.9 km] 15.775814148121055 -9.033333333333333
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 19:54 38.7602 43.4647 3.9 YEMLICE- (VAN) [North East  2.7 km] 6.3637927375337 -9.216666666666667
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 20:07 38.6813 43.4585 3.6 DILIMLI- (VAN) [North West  0.9 km] 6.055978090725582 -9.433333333333334
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 21:00 38.6798 43.2097 3.6 MOLLAKASIM- (VAN) [North East  1.8 km] 18.065081824506468 -10.316666666666666
#2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km] 2011.10.23 21:43 38.8012 43.3625 3.6 YAYLIYAKA- (VAN) [South West  1.9 km] 9.839498902616741 -11.033333333333333
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 12:32 38.7205 26.22 3.6 SALMAN ACIKLARI-IZMIR (EGE DENIZI) 16.74456425006754 -0.06666666666666667
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 12:35 38.8642 26.3445 3.9 EGE DENIZI 2.556630746288619 -0.11666666666666667
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 12:45 38.8877 26.3575 3.5 EGE DENIZI 5.338839573542808 -0.2833333333333333
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 12:47 38.8768 26.2742 3.8 EGE DENIZI 5.534145149821635 -0.31666666666666665
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 13:07 38.894 26.1123 3.6 MIDILLI ADASI (EGE DENIZI) 19.164048734296784 -0.65
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 13:10 38.9117 26.2815 3.5 EGE DENIZI 8.147885777901045 -0.7
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 13:42 38.8302 26.3123 3.6 EGE DENIZI 2.157651134361283 -1.2333333333333334
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 13:53 38.9397 26.2847 3.5 EGE DENIZI 10.908478777293942 -1.4166666666666667
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 13:57 38.909 26.2133 3.8 EGE DENIZI 11.902250711848305 -1.4833333333333334
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 14:19 38.8498 26.3388 4.4 EGE DENIZI 1.2240877047322432 -1.85
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 14:41 38.8815 26.3022 3.6 EGE DENIZI 4.342034553130121 -2.216666666666667
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 14:47 38.8615 26.3617 3.6 EGE DENIZI 3.5582869746142656 -2.316666666666667
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 15:25 38.8797 26.2488 4.0 EGE DENIZI 7.559053915313036 -2.95
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 16:16 38.7792 26.356 3.8 EGE DENIZI 7.976433840030022 -3.8
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 16:30 38.8327 26.336 4.2 EGE DENIZI 1.8256807226198568 -4.033333333333333
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 17:34 38.8322 26.3652 3.8 EGE DENIZI 3.8259361511227454 -5.1
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 18:25 38.8965 26.1522 4.2 MIDILLI ADASI (EGE DENIZI) 15.963921058357334 -5.95
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 18:29 38.8237 26.3188 4.0 EGE DENIZI 2.627742701493338 -6.016666666666667
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 19:48 39.0013 26.2978 3.7 MIDILLI ADASI (EGE DENIZI) 17.342386340161088 -7.333333333333333
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 19:52 38.936 26.2888 3.8 EGE DENIZI 10.406880768030618 -7.4
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 22:35 38.9327 26.4435 3.5 MIDILLI ADASI (EGE DENIZI) 14.002383588193958 -10.116666666666667
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.12 23:29 38.8873 26.3235 3.6 EGE DENIZI 4.505805200014102 -11.016666666666667
#2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI 2017.06.13 00:23 38.8547 26.3052 3.7 EGE DENIZI 1.9419734794413561 -11.916666666666666
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 22:34 36.9298 27.6018 3.7 GOKOVA KORFEZI (AKDENIZ) 17.971075765068022 -0.05
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 22:34 36.9663 27.4163 3.7 GOKOVA KORFEZI (AKDENIZ) 0.9990616300335782 -0.05
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 22:35 36.9237 27.4462 3.7 GOKOVA KORFEZI (AKDENIZ) 6.217996455467979 -0.06666666666666667
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 22:38 36.9532 27.3862 3.8 GOKOVA KORFEZI (AKDENIZ) 2.4913349060155747 -0.11666666666666667
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 22:39 36.9255 27.3333 3.6 GOKOVA KORFEZI (AKDENIZ) 8.069389784235527 -0.13333333333333333
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 22:40 36.9243 27.5655 3.8 GOKOVA KORFEZI (AKDENIZ) 15.05662817774461 -0.15
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 22:40 36.9253 27.5825 3.5 GOKOVA KORFEZI (AKDENIZ) 16.45563622631986 -0.15
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 22:44 36.9315 27.3987 3.6 GOKOVA KORFEZI (AKDENIZ) 4.248952844201756 -0.21666666666666667
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 22:48 36.9293 27.3467 3.7 GOKOVA KORFEZI (AKDENIZ) 6.8754236205517385 -0.2833333333333333
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 22:50 36.9088 27.5463 4.0 GOKOVA KORFEZI (AKDENIZ) 14.191682255304833 -0.31666666666666665
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 22:52 36.9503 27.372 4.5 GOKOVA KORFEZI (AKDENIZ) 3.6645992784643813 -0.35
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 22:54 36.9325 27.5875 3.8 GOKOVA KORFEZI (AKDENIZ) 16.665226800903227 -0.38333333333333336
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 23:00 36.9942 27.4098 4.0 BODRUM (MUGLA) [South West  4.8 km] 2.7926061146001535 -0.48333333333333334
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 23:09 36.8975 27.5877 3.5 GOKOVA KORFEZI (AKDENIZ) 18.039437695208463 -0.6333333333333333
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 23:14 36.9168 27.6 3.8 GOKOVA KORFEZI (AKDENIZ) 18.227699465269257 -0.7166666666666667
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 23:22 36.8792 27.5783 3.6 GOKOVA KORFEZI (AKDENIZ) 18.324241831765946 -0.85
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 23:23 36.958 27.3103 4.9 AKYARLAR-BODRUM (MUGLA) [South East  2.2 km] 8.568609939650743 -0.8666666666666667
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 23:45 36.9423 27.419 4.1 GOKOVA KORFEZI (AKDENIZ) 3.226487469691607 -1.2333333333333334
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 23:50 36.9417 27.4347 4.0 GOKOVA KORFEZI (AKDENIZ) 4.007334338806116 -1.3166666666666667
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.20 23:53 36.9132 27.5478 4.0 GOKOVA KORFEZI (AKDENIZ) 14.08547985122897 -1.3666666666666667
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 00:11 36.9362 27.4395 3.7 GOKOVA KORFEZI (AKDENIZ) 4.750496579814037 -1.6666666666666667
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 00:16 36.9742 27.3312 4.1 AKYARLAR-BODRUM (MUGLA) [East 3.4 km] 6.640770095018 -1.75
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 00:30 36.9108 27.6118 3.9 GOKOVA KORFEZI (AKDENIZ) 19.437746498028293 -1.9833333333333334
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 00:42 36.9398 27.3025 3.5 AKYARLAR-BODRUM (MUGLA) [South East  3.6 km] 9.739121335759712 -2.183333333333333
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 00:53 36.9158 27.3647 4.1 GOKOVA KORFEZI (AKDENIZ) 6.976144616845487 -2.3666666666666667
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 01:01 36.9727 27.3777 3.8 GOKOVA KORFEZI (AKDENIZ) 2.5160358999546606 -2.5
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 01:03 36.9205 27.5138 3.6 GOKOVA KORFEZI (AKDENIZ) 11.033296612719482 -2.533333333333333
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 01:17 36.9377 27.5835 4.0 GOKOVA KORFEZI (AKDENIZ) 16.18508057919998 -2.7666666666666666
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 01:25 36.9767 27.4223 4.1 BODRUM (MUGLA) [South 6.5 km] 1.6887092578393548 -2.9
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 01:35 36.9448 27.5775 4.4 GOKOVA KORFEZI (AKDENIZ) 15.50638377722351 -3.066666666666667
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 01:38 36.932 27.5367 4.6 GOKOVA KORFEZI (AKDENIZ) 12.3577204404789 -3.1166666666666667
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 01:50 36.9843 27.3765 4.2 YALI-BODRUM (MUGLA) [South East  5.2 km] 3.0838578611469614 -3.316666666666667
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 01:54 36.9508 27.3637 4.2 GOKOVA KORFEZI (AKDENIZ) 4.261172165156128 -3.3833333333333333
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 01:56 36.9332 27.3843 4.2 GOKOVA KORFEZI (AKDENIZ) 4.441794453134277 -3.4166666666666665
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 02:01 36.9402 27.575 3.6 GOKOVA KORFEZI (AKDENIZ) 15.387579340606598 -3.5
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 02:12 36.8833 27.3415 4.6 GOKOVA KORFEZI (AKDENIZ) 11.136148134821207 -3.683333333333333
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 02:54 36.9153 27.6098 3.5 GOKOVA KORFEZI (AKDENIZ) 19.106728624969172 -4.383333333333334
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 03:09 36.9383 27.5325 3.9 GOKOVA KORFEZI (AKDENIZ) 11.782722568776483 -4.633333333333334
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 03:54 36.9535 27.517 3.5 GOKOVA KORFEZI (AKDENIZ) 10.04379047741038 -5.383333333333334
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 03:55 36.9378 27.548 3.9 GOKOVA KORFEZI (AKDENIZ) 13.120736149599452 -5.4
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 03:59 36.9155 27.5858 4.4 GOKOVA KORFEZI (AKDENIZ) 17.087199582492374 -5.466666666666667
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 04:39 36.917 27.5877 3.9 GOKOVA KORFEZI (AKDENIZ) 17.18817338091801 -6.133333333333334
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 04:40 36.9107 27.4157 3.8 GOKOVA KORFEZI (AKDENIZ) 6.5763617816453435 -6.15
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 05:03 36.9198 27.5538 4.5 GOKOVA KORFEZI (AKDENIZ) 14.266077458974186 -6.533333333333333
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 05:13 36.9138 27.6107 4.3 GOKOVA KORFEZI (AKDENIZ) 19.23570846160309 -6.7
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 05:26 36.9435 27.5602 3.6 GOKOVA KORFEZI (AKDENIZ) 14.02468603761603 -6.916666666666667
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 05:43 36.9603 27.6012 3.6 GOKOVA KORFEZI (AKDENIZ) 17.398085514711628 -7.2
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 05:52 36.9805 27.3677 4.2 GOKOVA KORFEZI (AKDENIZ) 3.598081315070855 -7.35
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 05:55 36.8693 27.5923 3.9 GOKOVA KORFEZI (AKDENIZ) 19.97048384275205 -7.4
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 06:47 36.9688 27.3502 3.6 GOKOVA KORFEZI (AKDENIZ) 4.930959731206768 -8.266666666666667
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 07:05 36.923 27.4832 4.0 GOKOVA KORFEZI (AKDENIZ) 8.598775694351529 -8.566666666666666
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 07:11 36.9367 27.467 3.9 GOKOVA KORFEZI (AKDENIZ) 6.543003262361376 -8.666666666666666
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 07:28 36.8845 27.5545 4.1 GOKOVA KORFEZI (AKDENIZ) 16.243770213576447 -8.95
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 07:47 36.9183 27.5132 3.5 GOKOVA KORFEZI (AKDENIZ) 11.109876057718099 -9.266666666666667
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 08:03 36.964 27.5918 3.8 GOKOVA KORFEZI (AKDENIZ) 16.54423086827175 -9.533333333333333
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 08:34 36.9318 27.5437 3.6 GOKOVA KORFEZI (AKDENIZ) 12.952514040135418 -10.05
#2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ) 2017.07.21 08:58 36.9453 27.4698 3.5 GOKOVA KORFEZI (AKDENIZ) 6.2897685820242355 -10.45

Top ten:
1999.08.17 00:01 40.76 29.97 7.4 BASISKELE (KOCAELI) [North East  2.0 km]
2011.10.23 10:41 38.7212 43.411 7.2 YEMLICE- (VAN) [North West  1.5 km]
1999.11.12 16:57 40.74 31.21 7.2 UGUR- (DUZCE) [North East  0.3 km]
1992.03.13 17:18 39.72 39.63 6.8 GUNEBAKAN- (ERZINCAN) [South West  1.7 km]
2017.07.20 22:31 36.9693 27.4057 6.6 GOKOVA KORFEZI (AKDENIZ)
2008.07.15 03:26 35.7948 27.8798 6.4 AKDENIZ
2003.05.01 00:27 39.01 40.46 6.4 KURTULUS- (BINGOL) [South West  4.3 km]
2017.06.12 12:28 38.8468 26.3252 6.3 EGE DENIZI
1998.06.27 13:55 36.96 35.52 6.3 HASANBEYLI-SARICAM (ADANA) [South 10.1 km]
2011.04.01 13:29 35.5138 26.5798 6.2 AKDENIZ

List of foreshocks and aftershocks (within 20 km and in 24 hours):
1992.03.13 17:18 39.72 39.63 6.8 GUNEBAKAN- (ERZINCAN) [South West  1.7 km] 1992.03.14 01:24 39.63 39.69 4.6 KUCUKKADAGAN- (ERZINCAN) [South East  1.7 km] 11.248114686422076 -8.1
1998.06.27 13:55 36.96 35.52 6.3 HASANBEYLI-SARICAM (ADANA) [South 10.1 km] 1998.06.27 14:07 36.81 35.61 